# 1- Install TorchVision

In [63]:
!pip3 install torch torchvision torchaudio

# clone github repo for Yolo5

In [64]:
!git clone https://github.com/ultralytics/yolov5 

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [65]:
!cd yolov5 & pip install -r requirements.txt 

# import libraries

In [66]:
import torch
from matplotlib import pyplot as plt
import torchvision.transforms as T
import numpy as np
import cv2
from PIL import Image


# 2- Load Yolo5 Model

In [67]:
model = torch.hub.load('ultralytics/yolov5','yolov5s')

Using cache found in C:\Users\PMLS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-18 Python-3.12.4 torch-2.4.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [68]:
model

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

# Importing Monocular Depth Estimation model

In [69]:
!git clone https://huggingface.co/spaces/depth-anything/Depth-Anything-V2

Cloning into 'Depth-Anything-V2'...
Filtering content:  40% (2/5)
Filtering content:  60% (3/5), 2.08 MiB | 454.00 KiB/s
Filtering content:  80% (4/5), 7.17 MiB | 323.00 KiB/s
Filtering content: 100% (5/5), 16.43 MiB | 579.00 KiB/s
Filtering content: 100% (5/5), 28.66 MiB | 819.00 KiB/s, done.


In [70]:
!cd Depth-Anything-V2 & pip install -r requirements.txt

# Creating depth image

In [97]:
import cv2
import torch

from depth_anything_v2.dpt import DepthAnythingV2

model_depth_anything = DepthAnythingV2(encoder='vitl', features=256, out_channels=[256, 512, 1024, 1024])
model_depth_anything.load_state_dict(torch.load('checkpoints/depth_anything_v2_vitl.pth', map_location='cpu'))
model_depth_anything.eval()

raw_img = cv2.imread('img2.jpg')
depth = model_depth_anything.infer_image(raw_img) # H x W raw depth map
print(depth) # (H, W)


[[     90.129      89.758      89.387 ...      130.14       124.1      118.07]
 [     90.162      89.895      89.628 ...      131.02      127.65      124.28]
 [     90.195      90.032      89.869 ...      131.91       131.2       130.5]
 ...
 [     247.02      246.96      246.89 ...       350.3      350.26      350.21]
 [     247.31      247.15         247 ...      350.39      350.64      350.89]
 [     247.59      247.35      247.11 ...      350.48      351.03      351.57]]


# Normalize and store depth map

In [98]:
import cv2
import torch
import torchvision.transforms as T
from PIL import Image
import numpy as np

depth_map = depth.squeeze()

# Normalize the depth map for visualization
depth_map = cv2.normalize(depth_map, None, 0, 255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imwrite("depth_map2.jpg", depth_map)


True

# Using Yolo on same depth Image to create Boundary 

In [99]:
img = cv2.imread("img2.jpg")
results = model(img)

# Extract bounding boxes, labels, and confidences
boxes = results.xyxy[0].numpy()  # bounding boxes
labels = results.names  # class labels

C:\Users\PMLS/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [100]:
print(boxes)


[[     423.43      475.72      708.04      1019.1     0.28806          67]]


# Calculate Height and width of image

In [101]:
# Load depth map
depth_map = cv2.imread("depth_map2.jpg", cv2.IMREAD_GRAYSCALE).astype(float)  # Read depth map as grayscale
max_depth_cm = 1000
depth_map = (depth_map / 255.0) * max_depth_cm
print(depth_map)
# Assuming a focal length (in pixels)


sensor_width_mm = 9.96  # in mm
focal_length_mm = 3.85  # in mm
image_width_pixels = img.shape[1] 
# Calculate focal length in pixels
focal_length_pixels = (focal_length_mm * image_width_pixels) / sensor_width_mm

# Define function to calculate object size
def calculate_object_size_in_cm(box, depth_map, focal_length_pixels):
    x1, y1, x2, y2 = map(int, box[:4])
    
    # Ensure coordinates are within image bounds
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(depth_map.shape[1], x2), min(depth_map.shape[0], y2)
    
    # Calculate pixel dimensions
    width_pixels = x2 - x1
    height_pixels = y2 - y1
    
    # Calculate depth of object
    depth_of_object = np.median(depth_map[y1:y2, x1:x2]) # Depth in centimeters
    
    # Convert pixel dimensions to real-world dimensions
    width_cm = (width_pixels * depth_of_object) / focal_length_pixels
    height_cm = (height_pixels * depth_of_object) / focal_length_pixels
    
    return width_cm, height_cm, depth_of_object

# Calculate and print object sizes in centimeters
for box in boxes:
    width_cm, height_cm, depth_of_object = calculate_object_size_in_cm(box, depth_map, focal_length_pixels)
    print(f"Object at {box} has a width of {width_cm:.2f} cm, height of {height_cm:.2f} cm, and an average depth of: {depth_of_object:.2f} cm")


[[     86.275      86.275      86.275 ...      223.53      203.92      188.24]
 [     86.275      86.275      86.275 ...      227.45      215.69      203.92]
 [     86.275      86.275      86.275 ...      231.37      227.45      227.45]
 ...
 [     635.29      635.29      635.29 ...      996.08      996.08      996.08]
 [     635.29      635.29      635.29 ...      996.08      996.08        1000]
 [     635.29      635.29      635.29 ...      996.08        1000        1000]]
Object at [     423.43      475.72      708.04      1019.1     0.28806          67] has a width of 424.07 cm, height of 809.45 cm, and an average depth of: 690.20 cm


# Draw bounding boxes, width, height, and depth information on the original image

In [102]:

for box in boxes:
    x1, y1, x2, y2,_,_ = map(int, box)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    width_cm, height_cm, depth_of_object = calculate_object_size_in_cm(box, depth_map, focal_length_pixels)
    depth_text = f"Depth: {depth_of_object:.2f} cm" if not np.isnan(depth_of_object) else "Depth: N/A"
    dimension_text = f"W: {width_cm:.2f} cm, H: {height_cm:.2f} cm"
    cv2.putText(img, depth_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(img, dimension_text, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Save and show the output image
output_image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
output_image.save("output_image_with_real_dimensions2.jpg")
output_image.show()